# Assignment 2 - N-Grams Answer Key

In [1]:
import nltk
from nltk.corpus import udhr
import re
import collections
import pandas as pd
from nltk import word_tokenize
import random
import string
import collections

## Question 1 - Train and Test Set Creation

In [2]:
en_text = list(pd.read_csv('../data/CONcreTEXT_trial_EN.tsv',sep ='\t')['TEXT'])
it_text = list(pd.read_csv('../data/CONcreTEXT_trial_IT.tsv',sep='\t')['TEXT'])

In [4]:
#print(en_text)

In [5]:
print(en_text[:5])
random.shuffle(en_text)
random.shuffle(it_text)
print(en_text[:5])
en_words = []
it_words = []
for sent in en_text:
    for w in word_tokenize(sent):
        en_words.append(w)
    
        
for sent in it_text:
    for w in word_tokenize(sent):
        it_words.append(w)
print(u'# English words: {} \t # Italian Words: {}'.format(len(en_words),len(it_words)))

['Bring up academic achievements , awards , and other milestones in your life . ', 'Please list people you have helped , your personal achievements , or troublesome times you have overcome . ', 'Add activated carbon straight to your vodka . ', 'Place sensors around your garden , and when a cat comes in , the motion activates a sensor . ', 'Look for a partner that shares your level of adventure in pursuing new types of experiences . ']
['a loan with annual interest adds the interest rate ten times in ten years . ', "If the roses are out of water even for a few minutes , they 'll suffer for it . ", 'Look at the woman whom you are listening to for as long as you or she is speaking . ', "Hold the person 's hand when you cross the street . ", "It 's no good trying to chase a lizard away when he still has dozens of places to hide . "]
# English words: 1471 	 # Italian Words: 1471


In [4]:
en_train = [w.lower() for w in en_words if w not in string.punctuation][:int(len(en_words)*.8)]
en_test = [w.lower() for w in en_words if w not in string.punctuation][int(len(en_words)*.8):]

In [5]:
it_train = [w.lower() for w in it_words if w not in string.punctuation][:int(len(it_words)*.8)]
it_test = [w.lower() for w in it_words if w not in string.punctuation][int(len(it_words)*.8):]

## Question 2 - Unigram Language Model

In [6]:
nltk.FreqDist(nltk.bigrams(en_train))

FreqDist({('if', 'you'): 7, ('of', 'the'): 5, ('you', 'are'): 4, ('or', 'a'): 3, ('in', 'the'): 3, ('do', "n't"): 3, ('for', 'a'): 3, ('part', 'of'): 3, ('a', 'few'): 3, ('help', 'you'): 3, ...})

In [7]:
#en_train

In [28]:
def to_letters(train):
    letters = []
    for w in train:
        letters.append('<s>')
        for l in w:
            letters.append(l)
        letters.append('</s>')
    return letters

In [8]:
#to_letters(en_train)

In [30]:
def gen_gram_prob(train):
    train = to_letters(train)

    total_letters = len(train)
    unigrams = collections.defaultdict(lambda:0)
    uni_freq = nltk.FreqDist(train)
    """
    Unigram - p = c (w)/ N -> N = total number of letters (unigrams in corpus)
    """
    for k,v in uni_freq.items():
        unigrams[k] = v / float(total_letters)

    """
    Bigram - p = c{w,w2}/c(w)
    """

    bigrams = collections.defaultdict(lambda: collections.defaultdict(lambda: 0))
    bi_freq = nltk.FreqDist(nltk.bigrams(train))
    for k, v in bi_freq.items():
        bigrams[k] = v / uni_freq[k[0]]

    #print(bi)

    """
    Trigram - p = c(w,w2,w3)/c(w,w2)
    """

    trigrams = collections.defaultdict(lambda: collections.defaultdict(lambda: 0))
    tri_freq = nltk.FreqDist(nltk.trigrams(train))
    for k, v in tri_freq.items():
        trigrams[k] = v / bi_freq[(k[0], k[1])]
        
    return unigrams,bigrams,trigrams
    
    

In [64]:
def run_test (a_models,b_models,test_set):
    count = 0
    uni_a = a_models[0]
    uni_b = b_models [0]

    bi_a = a_models[1]
    bi_b = b_models[1]

    tri_a = a_models[2]
    tri_b = b_models[2]
    scores = [0, 0, 0]
    for word in test_set:

        word = word.lower()
        """
        0 - uni
        1 - bi
        2 - tri
        """
        a_probs = [1, 1, 1]
        b_probs = [1, 1, 1]



        for letter in word:
            """
           P(w) = P(c)*P(c-1)*P(c-n)
            """
            if letter in uni_a.keys() and uni_b.keys():
                a_probs[0] = a_probs[0]*uni_a[letter]
                b_probs[0] = b_probs[0]*uni_b[letter]

            elif letter in uni_a.keys():
                a_probs[0] = a_probs[0] * uni_a[letter]
                b_probs[0] = 0
            elif letter in uni_b.keys():
                a_probs[0] = 0
                b_probs [0] = b_probs [0] * uni_b[letter]
            else:
                # No smoothing, if letter is not seen in the data set treat it as 0
                a_probs[0] = 0
                b_probs [0] = 0
        if float(a_probs [0]) - float(b_probs [0]) > 0:
            # adds +1 if a_score is higher than b_score
            scores[0] += 1


        bi = nltk.bigrams(['<s>']+ [l for l in word]+ ['</s>'])
        tri = nltk.trigrams(['<s>']+ [l for l in word]+ ['</s>'])
        for bigram in bi:
            if bigram in bi_a.keys() and bigram in bi_b.keys():
                a_probs[1] = a_probs[1]* bi_a[bigram]
                b_probs[1] = b_probs[1] * bi_b[bigram]
            elif bigram in bi_a.keys():
                a_probs[1] = a_probs[1] * bi_a[bigram]
                b_probs [1] = 0
            elif bigram in bi_b.keys():
                a_probs[1] = 0
                b_probs[1] = b_probs[1] * bi_b[bigram]
            else:
                count += 1
                a_probs [1] = 0
                b_probs [1] = 0

        if a_probs [1] - b_probs [1] > 0:
            # adds +1 if a_score is higher than b_score
            scores[1] += 1



        for trigram in tri:
            if trigram in tri_a.keys() and trigram in tri_b.keys():
                a_probs[2] = a_probs[2]* tri_a[trigram]
                b_probs[2] = b_probs[2] * tri_b[trigram]
            elif trigram in tri_a.keys():
                a_probs[2] = a_probs[2] * tri_a[trigram]
                b_probs [2] = 0
            elif trigram in tri_b.keys():
                a_probs[2] = 0
                b_probs[2] = b_probs[2] * tri_b[trigram]
            else:
                a_probs [2] = 0
                b_probs [2] = 0

        if a_probs [2] - b_probs [2] > 0:
            # adds +1 if a_score is higher than b_score
            scores[2] += 1




    scores [0] = scores[0] / len(test_set)
    scores [1] = scores[1] / len(test_set)
    scores [2] = scores[2] / len(test_set)

    print(count)

    return scores

In [65]:
uni_it, bi_it, tri_it = gen_gram_prob(it_train)
uni_eng,bi_eng,tri_eng = gen_gram_prob(en_train)

## Question 2 - Unigram Accuracy Score

In [69]:
eng_scores = run_test([uni_eng,bi_eng,tri_eng],[uni_it, bi_it, tri_it],en_test)
print (u'Unigrams English Vs. Italian: {} %'.format(round(eng_scores[0]*100,2)))

5
Unigrams English Vs. Italian: 51.8 %


## Question 3 - Bigram Accuracy Score

In [70]:
print (u'Bigrams English Vs. Italian: {} %'.format(round(eng_scores[1]*100,2)))

Bigrams English Vs. Italian: 77.7 %


## Question 4 - Trigram Accuracy Score

In [71]:
print (u'Trigrams English Vs. Italian: {} %'.format(round(eng_scores[2]*100,2)))

Trigrams English Vs. Italian: 66.19 %
